***step -1 load the dataset***

In [15]:
import pandas as pd 

df_liver = pd.read_csv(r'c:\Users\Thennarasu\OneDrive\Documents\dataset\indian_liver_patient - indian_liver_patient.csv')

***Step 2  Clean the data***


In [16]:
df_liver

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
0,65,Female,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,Male,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1
...,...,...,...,...,...,...,...,...,...,...,...
578,60,Male,0.5,0.1,500,20,34,5.9,1.6,0.37,2
579,40,Male,0.6,0.1,98,35,31,6.0,3.2,1.10,1
580,52,Male,0.8,0.2,245,48,49,6.4,3.2,1.00,1
581,31,Male,1.3,0.5,184,29,32,6.8,3.4,1.00,1


In [17]:
df_liver.isnull().sum()

Age                           0
Gender                        0
Total_Bilirubin               0
Direct_Bilirubin              0
Alkaline_Phosphotase          0
Alamine_Aminotransferase      0
Aspartate_Aminotransferase    0
Total_Protiens                0
Albumin                       0
Albumin_and_Globulin_Ratio    4
Dataset                       0
dtype: int64

In [18]:
print(df_liver.duplicated().sum())

13


In [19]:
df_liver = df_liver.drop_duplicates()


In [20]:
df_liver['Albumin_and_Globulin_Ratio'].mean()


np.float64(0.9480035335689044)

In [ ]:
df_liver['Albumin_and_Globulin_Ratio'] = df_liver['Albumin_and_Globulin_Ratio'].fillna(0.9480035)


C:\Users\Thennarasu\AppData\Local\Temp\ipykernel_3864\2074778549.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_liver['Albumin_and_Globulin_Ratio'] = df_liver['Albumin_and_Globulin_Ratio'].fillna(0.947064)


### 👉 Change labels:

2 → 1 (Disease)

1 → 0 (No Disease) 

In [22]:
df_liver['Dataset'] = df_liver['Dataset'].replace([2, 1], [1, 0])


C:\Users\Thennarasu\AppData\Local\Temp\ipykernel_3864\1372367821.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_liver['Dataset'] = df_liver['Dataset'].replace([2, 1], [1, 0])


### 👉 Convert Gender (Male/Female) into numbers (0/1)

In [23]:
from sklearn.preprocessing import OrdinalEncoder
df_liver['Gender'] = OrdinalEncoder().fit_transform(df_liver[['Gender']])


C:\Users\Thennarasu\AppData\Local\Temp\ipykernel_3864\2622480587.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_liver['Gender'] = OrdinalEncoder().fit_transform(df_liver[['Gender']])


***Step 3 – Split into Features & Target***

In [24]:
X = df_liver.drop(columns=['Dataset'])  
y = df_liver['Dataset']


## class imbalance ##

In [25]:
df_liver['Dataset'].value_counts(normalize=True) * 100

Dataset
0    71.22807
1    28.77193
Name: proportion, dtype: float64

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix,roc_auc_score
from imblearn.over_sampling import SMOTE

### 👉 Train with 80% data, test with 20% data

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [34]:
# Apply SMOTE for balancing
sm = SMOTE(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X, y)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

## build a model

In [ ]:
model = RandomForestClassifier(class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', random_state=42)

In [ ]:
# Predict
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]   # probabilities for ROC

In [ ]:


print("\n📊 Classification Report:\n",   classification_report(y_test, y_pred))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("\n📌 Confusion Matrix:")
print(cm)

# ROC-AUC Score
roc_auc = roc_auc_score(y_test, y_prob)
print("\n⭐ ROC-AUC Score:", round(roc_auc, 3))


📊 Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.83      0.83        75
           1       0.86      0.86      0.86        92

    accuracy                           0.84       167
   macro avg       0.84      0.84      0.84       167
weighted avg       0.84      0.84      0.84       167


📌 Confusion Matrix:
[[62 13]
 [13 79]]

⭐ ROC-AUC Score: 0.919


## save model 

In [1]:
import pickle

In [ ]:
with open("liver.pkl", "wb") as f:
    pickle.dump(model, f)